In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fancyimpute import KNN
from sklearn.preprocessing import OrdinalEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
Using TensorFlow backend.


In [2]:
Train = pd.read_csv('/content/Train_aBjfeNk.csv')
Test = pd.read_csv('/content/Test_LqhgPWU.csv')
Sample_submission = pd.read_csv('/content/sample_submission_wyi0h0z.csv')

Train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
ID_main = Test['ID']
Test = Test.drop(labels='ID', axis=1)

In [4]:
Train.isnull().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [5]:
pd.options.display.max_columns = None

In [6]:
Train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [7]:
# Categorical Columns to be encoded ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']
# Categorical Clomuns to be Imputed ['Ever_Married','Graduated','Profession','Work_Experience','Family_Size','Var_1']

In [8]:
impute_data = pd.read_csv('/content/Train_aBjfeNk.csv')
impute_data = impute_data.drop(labels='Segmentation', axis=1)

In [9]:
#instantiate both packages to use
encoder = OrdinalEncoder()
imputer = KNN()
# create a list of categorical columns to iterate over
cat_cols = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(impute_data[columns])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setit

In [10]:
impute_data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,462809,1.0,0,22,0,5,1.0,2.0,4.0,3
1,462643,0.0,1,38,1,2,NaN,0.0,3.0,3
2,466315,0.0,1,67,1,2,1.0,2.0,1.0,5
3,461735,1.0,1,67,1,7,0.0,1.0,2.0,5
4,462669,0.0,1,40,1,3,NaN,1.0,6.0,5
...,...,...,...,...,...,...,...,...,...,...
8063,464018,1.0,0,22,0,NaN,0.0,2.0,7.0,0
8064,464685,1.0,0,35,0,4,3.0,2.0,4.0,3
8065,465406,0.0,0,33,1,5,1.0,2.0,1.0,5
8066,467299,0.0,0,27,1,5,1.0,2.0,4.0,5


In [11]:
encode_data = pd.DataFrame(np.round(imputer.fit_transform(impute_data)),columns = impute_data.columns)

Imputing row 1/8068 with 0 missing, elapsed time: 11.229
Imputing row 101/8068 with 0 missing, elapsed time: 11.231
Imputing row 201/8068 with 0 missing, elapsed time: 11.233
Imputing row 301/8068 with 1 missing, elapsed time: 11.235
Imputing row 401/8068 with 0 missing, elapsed time: 11.237
Imputing row 501/8068 with 0 missing, elapsed time: 11.239
Imputing row 601/8068 with 0 missing, elapsed time: 11.241
Imputing row 701/8068 with 0 missing, elapsed time: 11.243
Imputing row 801/8068 with 0 missing, elapsed time: 11.245
Imputing row 901/8068 with 0 missing, elapsed time: 11.248
Imputing row 1001/8068 with 0 missing, elapsed time: 11.250
Imputing row 1101/8068 with 0 missing, elapsed time: 11.252
Imputing row 1201/8068 with 0 missing, elapsed time: 11.253
Imputing row 1301/8068 with 0 missing, elapsed time: 11.255
Imputing row 1401/8068 with 0 missing, elapsed time: 11.257
Imputing row 1501/8068 with 0 missing, elapsed time: 11.259
Imputing row 1601/8068 with 1 missing, elapsed time:

In [12]:
encode_data.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

In [13]:
yTrain = Train['Segmentation']
xTrain = encode_data.drop(labels='ID', axis=1)

yTrain.shape, xTrain.shape

((8068,), (8068, 9))

In [14]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

X_train, X_valid, y_train, y_valid = train_test_split(xTrain,yTrain, test_size=0.3,random_state=1) 

In [15]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_valid)

acc = accuracy_score(y_pred, y_valid)

In [16]:
acc

0.5373812474184222

In [17]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred1 = xgb.predict(X_valid)

acc1 = accuracy_score(y_pred1, y_valid)

acc1


0.5373812474184222

In [18]:
encode_data.corr()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
ID,1.000000,0.006155,0.020881,-0.005055,-0.012452,-0.001738,-0.026622,-0.018949,0.012567,-0.002053
Gender,0.006155,1.000000,0.114073,0.030216,-0.034145,0.025572,-0.060100,-0.054261,0.054983,0.030552
Ever_Married,0.020881,0.114073,1.000000,0.569606,0.200540,-0.141169,-0.099494,-0.626437,-0.081666,0.085389
Age,-0.005055,0.030216,0.569606,1.000000,0.236602,0.039755,-0.196658,-0.269544,-0.284784,0.168863
Graduated,-0.012452,-0.034145,0.200540,0.236602,1.000000,-0.298892,0.025257,-0.153570,-0.228522,0.126231
Profession,-0.001738,0.025572,-0.141169,0.039755,-0.298892,1.000000,-0.022210,0.230166,0.086263,-0.042973
Work_Experience,-0.026622,-0.060100,-0.099494,-0.196658,0.025257,-0.022210,1.000000,0.049236,-0.055549,0.017256
Spending_Score,-0.018949,-0.054261,-0.626437,-0.269544,-0.153570,0.230166,0.049236,1.000000,-0.111089,-0.033157
Family_Size,0.012567,0.054983,-0.081666,-0.284784,-0.228522,0.086263,-0.055549,-0.111089,1.000000,-0.144522
Var_1,-0.002053,0.030552,0.085389,0.168863,0.126231,-0.042973,0.017256,-0.033157,-0.144522,1.000000


In [19]:
#instantiate both packages to use
encoder = OrdinalEncoder()
imputer = KNN()
# create a list of categorical columns to iterate over
cat_cols = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']

def encode(test_data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(test_data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    test_data.loc[test_data.notnull()] = np.squeeze(impute_ordinal)
    return test_data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(Test[columns])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setit

In [20]:
encode_test_data = pd.DataFrame(np.round(imputer.fit_transform(Test)),columns = Test.columns)

Imputing row 1/2627 with 0 missing, elapsed time: 1.195
Imputing row 101/2627 with 0 missing, elapsed time: 1.198
Imputing row 201/2627 with 0 missing, elapsed time: 1.202
Imputing row 301/2627 with 0 missing, elapsed time: 1.204
Imputing row 401/2627 with 1 missing, elapsed time: 1.205
Imputing row 501/2627 with 1 missing, elapsed time: 1.207
Imputing row 601/2627 with 0 missing, elapsed time: 1.209
Imputing row 701/2627 with 0 missing, elapsed time: 1.211
Imputing row 801/2627 with 0 missing, elapsed time: 1.212
Imputing row 901/2627 with 0 missing, elapsed time: 1.214
Imputing row 1001/2627 with 0 missing, elapsed time: 1.215
Imputing row 1101/2627 with 0 missing, elapsed time: 1.217
Imputing row 1201/2627 with 1 missing, elapsed time: 1.218
Imputing row 1301/2627 with 0 missing, elapsed time: 1.220
Imputing row 1401/2627 with 0 missing, elapsed time: 1.222
Imputing row 1501/2627 with 0 missing, elapsed time: 1.223
Imputing row 1601/2627 with 0 missing, elapsed time: 1.225
Imputing 

In [28]:
Segmentation = pd.Series(np.array(xgb.predict(encode_test_data)), name="Segmentation")

results = pd.concat([ID_main,Segmentation],axis=1)

results.to_csv("submission.csv",index=False)